In [1]:
import pandas as pd 
import numpy as np

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid

In [5]:
class HyperParamClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, param_grid):
        self.param_grid = param_grid
        self.model = None  # Placeholder for the trained model

    def fit(self, X, y, n_estimators=5):
        # Generate all possible hyperparameter combinations
        all_param_combos = list(ParameterGrid(self.param_grid))

        # Shuffle and select `n_estimators` random hyperparameter combinations
        np.random.shuffle(all_param_combos)
        chosen_combos = all_param_combos[:n_estimators]

        # Store all trained models
        self.estimators_ = []

        # Train an estimator for each selected hyperparameter combination
        for params in chosen_combos:
            model = DecisionTreeClassifier(**params)  # Initialize Decision Tree
            model.fit(X, y)  # Train on the dataset
            self.estimators_.append(model)  # Store the trained model
    
        return self

    def predict(self, X):
        # Ensure the model has been trained before making predictions
        if self.model is None:
            raise ValueError("Model has not been trained. Call `fit` first.")

        # Use the trained DecisionTreeClassifier to make predictions
        return self.model.predict(X)


    def predict_proba(self, X):
        if self.model is None:
            raise ValueError("Model has not been trained. Call `fit` first.")
        
        return self.model.predict_proba(X)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
# Define hyperparameter grid
param_grid = {
    'max_depth': [5, 10, 15], 
    'min_samples_split': [2, 5, 10]
}

data = pd.read_csv("mnist_train.csv")

X = data.drop("label",axis = 1)  # X = images, y = digit labels
y = data["label"]
# Convert labels to integers
y = y.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize classifier
clf = HyperParamClassifier(param_grid)

# Train ensemble with 3 decision trees
clf.fit(X_train, y_train, n_estimators=3)

# Check trained models
print("Number of trained models:", len(clf.estimators_))
for i, model in enumerate(clf.estimators_):
    print(f"Model {i+1} hyperparameters:", model.get_params())
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    print(f"Model Accuracy on MNIST: {accuracy:.4f}")


Number of trained models: 3
Model 1 hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
Model Accuracy on MNIST: 0.8581
Model 2 hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}
Model Accuracy on MNIST: 0.6579
Model 3 hyperparameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 15, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0,